In [103]:
import multiprocessing
from support.utilities import *
from support.language_models import *
from support.norms import *
from cipher.keyword_cipher import *
from cipher.polybius import *

In [23]:
g = polybius_grid('playfair example', [1,2,3,4,5], [1,2,3,4,5])
g

{'p': (1, 1),
 'l': (1, 2),
 'a': (1, 3),
 'y': (1, 4),
 'f': (1, 5),
 'i': (2, 1),
 'r': (2, 2),
 'e': (2, 3),
 'x': (2, 4),
 'm': (2, 5),
 'b': (3, 1),
 'c': (3, 2),
 'd': (3, 3),
 'g': (3, 4),
 'h': (3, 5),
 'k': (4, 1),
 'n': (4, 2),
 'o': (4, 3),
 'q': (4, 4),
 's': (4, 5),
 't': (5, 1),
 'u': (5, 2),
 'v': (5, 3),
 'w': (5, 4),
 'z': (5, 5),
 'j': (2, 1)}

In [14]:
def playfair_wrap(n, lowest, highest):
    skip = highest - lowest + 1
    while n > highest or n < lowest:
        if n > highest:
            n -= skip
        if n < lowest:
            n += skip
    return n

In [20]:
playfair_wrap(11, 1, 5)

1

In [66]:
def playfair_encipher_bigram(ab, grid, padding_letter='x'):
    a, b = ab
    max_row = max(c[0] for c in grid.values())
    max_col = max(c[1] for c in grid.values())
    min_row = min(c[0] for c in grid.values())
    min_col = min(c[1] for c in grid.values())
    if a == b:
        b = padding_letter
    if grid[a][0] == grid[b][0]:  # same row
        cp = (grid[a][0], playfair_wrap(grid[a][1] + 1, min_col, max_col))
        dp = (grid[b][0], playfair_wrap(grid[b][1] + 1, min_col, max_col))
    elif grid[a][1] == grid[b][1]:  # same column
        cp = (playfair_wrap(grid[a][0] + 1, min_row, max_row), grid[a][1])
        dp = (playfair_wrap(grid[b][0] + 1, min_row, max_row), grid[b][1])
    else:
        cp = (grid[a][0], grid[b][1])
        dp = (grid[b][0], grid[a][1])
    c = [k for k, v in grid.items() if v == cp][0]
    d = [k for k, v in grid.items() if v == dp][0]
    return c + d

In [68]:
playfair_encipher_bigram('ex', g)

'xm'

In [69]:
def playfair_decipher_bigram(ab, grid, padding_letter='x'):
    a, b = ab
    max_row = max(c[0] for c in grid.values())
    max_col = max(c[1] for c in grid.values())
    min_row = min(c[0] for c in grid.values())
    min_col = min(c[1] for c in grid.values())
    if a == b:
        b = padding_letter
    if grid[a][0] == grid[b][0]:  # same row
        cp = (grid[a][0], playfair_wrap(grid[a][1] - 1, min_col, max_col))
        dp = (grid[b][0], playfair_wrap(grid[b][1] - 1, min_col, max_col))
    elif grid[a][1] == grid[b][1]:  # same column
        cp = (playfair_wrap(grid[a][0] - 1, min_row, max_row), grid[a][1])
        dp = (playfair_wrap(grid[b][0] - 1, min_row, max_row), grid[b][1])
    else:
        cp = (grid[a][0], grid[b][1])
        dp = (grid[b][0], grid[a][1])
    c = [k for k, v in grid.items() if v == cp][0]
    d = [k for k, v in grid.items() if v == dp][0]
    return c + d

In [70]:
playfair_decipher_bigram('xm', g)

'ex'

In [36]:
chunks(sanitise('hide the gold in the tree stump'), 2)

['hi', 'de', 'th', 'eg', 'ol', 'di', 'nt', 'he', 'tr', 'ee', 'st', 'um', 'p']

In [75]:
def playfair_bigrams(text, padding_letter='x', padding_replaces_repeat=True):
    i = 0
    bigrams = []
    while i < len(text):
        bigram = text[i:i+2]
        if len(bigram) == 1:
            i = len(text) + 1
            bigram = bigram + padding_letter
        else:
            if bigram[0] == bigram[1]:
                bigram = bigram[0] + padding_letter
                if padding_replaces_repeat:
                    i += 2
                else:
                    i += 1
            else:
                i += 2
        bigrams += [bigram]
    return bigrams
    

In [76]:
playfair_bigrams(sanitise('hide the gold in the tree stump'), padding_replaces_repeat=True)

['hi', 'de', 'th', 'eg', 'ol', 'di', 'nt', 'he', 'tr', 'ex', 'st', 'um', 'px']

In [77]:
playfair_bigrams(sanitise('hide the gold in the tree stump'), padding_replaces_repeat=False)

['hi', 'de', 'th', 'eg', 'ol', 'di', 'nt', 'he', 'tr', 'ex', 'es', 'tu', 'mp']

In [73]:
ct = cat(playfair_encipher_bigram((b[0], b[1]), g) for b in playfair_bigrams(sanitise('hide the gold in the tree stump')))
ct

'bmodzbxdnabekudmuixmmouvif'

In [74]:
cat(playfair_decipher_bigram((b[0], b[1]), g) for b in playfair_bigrams(sanitise(ct)))

'hidethegoldinthetrexestump'

In [128]:
def playfair_encipher(message, keyword, padding_letter='x',
                      padding_replaces_repeat=False,
#                       column_order=None, row_order=None, 
#                       column_first=False, 
                      letters_to_merge=None, 
                      wrap_alphabet=KeywordWrapAlphabet.from_a):
    column_order = list(range(5))
    row_order = list(range(5))
    if letters_to_merge is None: 
        letters_to_merge = {'j': 'i'}   
    grid = polybius_grid(keyword, column_order, row_order,
                        letters_to_merge=letters_to_merge,
                        wrap_alphabet=wrap_alphabet)
    message_bigrams = playfair_bigrams(sanitise(message), padding_letter=padding_letter, 
                                       padding_replaces_repeat=padding_replaces_repeat)
    ciphertext_bigrams = [playfair_encipher_bigram(b, grid, padding_letter=padding_letter) for b in message_bigrams]
    return cat(ciphertext_bigrams)

In [129]:
def playfair_decipher(message, keyword, padding_letter='x',
                      padding_replaces_repeat=False,
#                       column_order=None, row_order=None, 
#                       column_first=False, 
                      letters_to_merge=None, 
                      wrap_alphabet=KeywordWrapAlphabet.from_a):
    column_order = list(range(5))
    row_order = list(range(5))
    if letters_to_merge is None: 
        letters_to_merge = {'j': 'i'}   
    grid = polybius_grid(keyword, column_order, row_order,
                        letters_to_merge=letters_to_merge,
                        wrap_alphabet=wrap_alphabet)
    message_bigrams = playfair_bigrams(sanitise(message), padding_letter=padding_letter, 
                                       padding_replaces_repeat=padding_replaces_repeat)
    plaintext_bigrams = [playfair_decipher_bigram(b, grid, padding_letter=padding_letter) for b in message_bigrams]
    return cat(plaintext_bigrams)

In [130]:
prr = True
plaintext = 'hide the gold in the tree stump'
key = 'playfair example'
ciphertext = playfair_encipher(plaintext, key, padding_replaces_repeat=prr)
recovered_plaintext = playfair_decipher(ciphertext, key, padding_replaces_repeat=prr)
ciphertext, recovered_plaintext

('bmodzbxdnabekudmuixmkzzryi', 'hidethegoldinthetrexstumpx')

In [131]:
prr = False
plaintext = 'hide the gold in the tree stump'
key = 'playfair example'
ciphertext = playfair_encipher(plaintext, key, padding_replaces_repeat=prr)
recovered_plaintext = playfair_decipher(ciphertext, key, padding_replaces_repeat=prr)
ciphertext, recovered_plaintext

('bmodzbxdnabekudmuixmmouvif', 'hidethegoldinthetrexestump')

In [132]:
prr = False
plaintext = 'hide the gold in the tree stump'
key = 'simple key'
ciphertext = playfair_encipher(plaintext, key, padding_replaces_repeat=prr)
recovered_plaintext = playfair_decipher(ciphertext, key, padding_replaces_repeat=prr)
ciphertext, recovered_plaintext

('dlckztactiokoncbntaucenzpl', 'hidethegoldinthetrexestump')

In [134]:
def playfair_break_mp(message, 
                      letters_to_merge=None, padding_letter='x',
                      wordlist=keywords, fitness=Pletters,
                      number_of_solutions=1, chunksize=500):
    if letters_to_merge is None: 
        letters_to_merge = {'j': 'i'}   

    with multiprocessing.Pool() as pool:
        helper_args = [(message, word, wrap, 
                        letters_to_merge, padding_letter,
                        pad_replace,
                        fitness)
                       for word in wordlist
                       for wrap in KeywordWrapAlphabet
                       for pad_replace in [False, True]]
        # Gotcha: the helper function here needs to be defined at the top level
        #   (limitation of Pool.starmap)
        breaks = pool.starmap(playfair_break_worker, helper_args, chunksize)
        if number_of_solutions == 1:
            return max(breaks, key=lambda k: k[1])
        else:
            return sorted(breaks, key=lambda k: k[1], reverse=True)[:number_of_solutions]

In [135]:
def playfair_break_worker(message, keyword, wrap, 
                          letters_to_merge, padding_letter,
                          pad_replace,
                          fitness):
    plaintext = playfair_decipher(message, keyword, padding_letter,
                                  pad_replace,
                                  letters_to_merge, 
                                  wrap)
    if plaintext:
        fit = fitness(plaintext)
    else:
        fit = float('-inf')
    logger.debug('Playfair break attempt using key {0} (wrap={1}, merging {2}, '
                 'pad replaces={3}), '
                 'gives fit of {4} and decrypt starting: '
                 '{5}'.format(keyword, wrap, letters_to_merge, pad_replace,
                              fit, sanitise(plaintext)[:50]))
    return (keyword, wrap, letters_to_merge, padding_letter, pad_replace), fit

In [136]:
playfair_break_mp(playfair_encipher('this is a test message for the ' \
          'polybius decipherment', 'elephant'), \
          wordlist=['cat', 'elephant', 'kangaroo']) 

(('elephant', <KeywordWrapAlphabet.from_a: 1>, {'j': 'i'}, 'x', False),
 -54.53880323982303)

In [200]:
def playfair_simulated_annealing_break(message, workers=10, 
                              initial_temperature=200,
                              max_iterations=20000,
                              plain_alphabet=None, 
                              cipher_alphabet=None, 
                              fitness=Pletters, chunksize=1):
    worker_args = []
    ciphertext = sanitise(message)
    for i in range(workers):
        if plain_alphabet is None:
            used_plain_alphabet = string.ascii_lowercase
        else:
            used_plain_alphabet = plain_alphabet
        if cipher_alphabet is None:
#             used_cipher_alphabet = list(string.ascii_lowercase)
#             random.shuffle(used_cipher_alphabet)
#             used_cipher_alphabet = cat(used_cipher_alphabet)
            used_cipher_alphabet = random.choice(keywords)
        else:
            used_cipher_alphabet = cipher_alphabet
        worker_args.append((ciphertext, used_plain_alphabet, used_cipher_alphabet, 
                            initial_temperature, max_iterations, fitness))
    with multiprocessing.Pool() as pool:
        breaks = pool.starmap(playfair_simulated_annealing_break_worker,
                              worker_args, chunksize)
    return max(breaks, key=lambda k: k[1])

In [205]:
def playfair_simulated_annealing_break_worker(message, plain_alphabet, cipher_alphabet, 
                                     t0, max_iterations, fitness):
    def swap(letters, i, j):
        if i > j:
            i, j = j, i
        if i == j:
            return letters
        else:
            return (letters[:i] + letters[j] + letters[i+1:j] + letters[i] +
                    letters[j+1:])
    
    temperature = t0

    dt = t0 / (0.9 * max_iterations)
    
    current_alphabet = cipher_alphabet
#     current_wrap = KeywordWrapAlphabet.from_a
    current_letters_to_merge = {'j': 'i'}
    current_pad_replace = False
    current_padding_letter = 'x'
    
    alphabet = current_alphabet
#     wrap = current_wrap
    letters_to_merge = current_letters_to_merge
    pad_replace = current_pad_replace
    padding_letter = current_padding_letter
    plaintext = playfair_decipher(message, alphabet, padding_letter,
                                  pad_replace,
                                  letters_to_merge, 
                                  KeywordWrapAlphabet.from_a)
    current_fitness = fitness(plaintext)

    best_alphabet = current_alphabet
#     best_wrap = current_wrap
    best_letters_to_merge = current_letters_to_merge
    best_pad_replace = current_pad_replace
    best_padding_letter = current_padding_letter
    best_fitness = current_fitness
    best_plaintext = plaintext
    
    # print('starting for', max_iterations)
    for i in range(max_iterations):
        chosen = random.random()
#         if chosen < 0.7:
#             swap_a = random.randrange(26)
#             swap_b = (swap_a + int(random.gauss(0, 4))) % 26
#             alphabet = swap(current_alphabet, swap_a, swap_b)
# #         elif chosen < 0.8:
# #             wrap = random.choice(list(KeywordWrapAlphabet))
#         elif chosen < 0.8:
#             pad_replace = random.choice([True, False])
#         elif chosen < 0.9:
#             letter_from = random.choice(string.ascii_lowercase)
#             letter_to = random.choice([c for c in string.ascii_lowercase if c != letter_from])
#             letters_to_merge = {letter_from: letter_to}
#         else:
#             padding_letter = random.choice(string.ascii_lowercase)

        if chosen < 0.7:
            swap_a = random.randrange(len(current_alphabet))
            swap_b = (swap_a + int(random.gauss(0, 4))) % len(current_alphabet)
            alphabet = swap(current_alphabet, swap_a, swap_b)
        elif chosen < 0.85:
            new_letter = random.choice(string.ascii_lowercase)
            alphabet = swap(current_alphabet + new_letter, random.randrange(len(current_alphabet)), len(current_alphabet))
        else:
            if len(current_alphabet) > 1:
                deletion_position = random.randrange(len(current_alphabet))
                alphabet = current_alphabet[:deletion_position] + current_alphabet[deletion_position+1:]
            else:
                alphabet = current_alphabet

        try:
            plaintext = playfair_decipher(message, alphabet, padding_letter,
                                  pad_replace,
                                  letters_to_merge, 
                                  KeywordWrapAlphabet.from_a)
        except:
            print("Error", alphabet, padding_letter,
                                  pad_replace,
                                  letters_to_merge)
            raise

        new_fitness = fitness(plaintext)
        try:
            sa_chance = math.exp((new_fitness - current_fitness) / temperature)
        except (OverflowError, ZeroDivisionError):
            # print('exception triggered: new_fit {}, current_fit {}, temp {}'.format(new_fitness, current_fitness, temperature))
            sa_chance = 0
        if (new_fitness > current_fitness or random.random() < sa_chance):
            # logger.debug('Simulated annealing: iteration {}, temperature {}, '
            #     'current alphabet {}, current_fitness {}, '
            #     'best_plaintext {}'.format(i, temperature, current_alphabet, 
            #     current_fitness, best_plaintext[:50]))

            # logger.debug('new_fit {}, current_fit {}, temp {}, sa_chance {}'.format(new_fitness, current_fitness, temperature, sa_chance))
            current_fitness = new_fitness
            current_alphabet = alphabet
#             current_wrap = wrap
            current_letters_to_merge = letters_to_merge
            current_pad_replace = pad_replace
            current_padding_letter = padding_letter
            
        if current_fitness > best_fitness:
            best_alphabet = current_alphabet
#             best_wrap = current_wrap
            best_letters_to_merge = current_letters_to_merge
            best_pad_replace = current_pad_replace
            best_padding_letter = current_padding_letter
            best_fitness = current_fitness
            best_plaintext = plaintext
        if i % 500 == 0:
            logger.debug('Simulated annealing: iteration {}, temperature {}, '
                'current alphabet {}, current_fitness {}, '
                'best_plaintext {}'.format(i, temperature, current_alphabet, 
                current_fitness, plaintext[:50]))
        temperature = max(temperature - dt, 0.001)

    print(best_alphabet, best_plaintext[:50])
    return { 'alphabet': best_alphabet
#            , 'wrap': best_wrap
           , 'letters_to_merge': best_letters_to_merge
           , 'pad_replace': best_pad_replace
           , 'padding_letter': best_padding_letter
           }, best_fitness # current_alphabet, current_fitness

In [149]:
ciphertext

'dlckztactiokoncbntaucenzpl'

In [168]:
key, score = playfair_simulated_annealing_break(ciphertext, fitness=Ptrigrams)
key, score

({'alphabet': 'ipknagqxvszjrmhlfyeutwdcbo',
  'wrap': <KeywordWrapAlphabet.from_largest: 3>,
  'letters_to_merge': {'x': 'z'},
  'pad_replace': False,
  'padding_letter': 't'},
 -85.75243058399522)

In [169]:
# polybius_grid(key['alphabet'], [1,2,3,4,5], [1,2,3,4,5], letters_to_merge=key['letters_to_merge'], wrap_alphabet=key['wrap'])

In [170]:
playfair_decipher(ciphertext, key['alphabet'], key['padding_letter'], key['pad_replace'], key['letters_to_merge'], key['wrap'])

'orecalkofacabadcauntemasar'

In [171]:
prr = False
plaintext = 'hide the gold in the tree stump'
key = 'simple'
ciphertext = playfair_encipher(plaintext, key, padding_replaces_repeat=prr)
recovered_plaintext = playfair_decipher(ciphertext, key, padding_replaces_repeat=prr)
ciphertext, recovered_plaintext

('gmearkafusalkufbutbvfeuopl', 'hidethegoldinthetrexestump')

In [174]:
playfair_break_mp(ciphertext, fitness=Ptrigrams)

(('simple', <KeywordWrapAlphabet.from_a: 1>, {'j': 'i'}, 'x', False),
 -80.29349856508469)

In [175]:
playfair_decipher(ciphertext, 'simple')

'hidethegoldinthetrexestump'

In [179]:
key, score = playfair_simulated_annealing_break(ciphertext, fitness=Ptrigrams, workers=50, max_iterations=int(1e5))
key, score

({'alphabet': 'rhbylupiwzevjdxfakcqtnomgs',
  'wrap': <KeywordWrapAlphabet.from_a: 1>,
  'letters_to_merge': {'p': 'f'},
  'pad_replace': True,
  'padding_letter': 'a'},
 -78.01490096572304)

In [180]:
playfair_decipher(ciphertext, key['alphabet'], key['padding_letter'], key['pad_replace'], key['letters_to_merge'], key['wrap'])

'mouthatventraidleardelines'

In [184]:
plaintext = open('2018/5b.plaintext').read()
plaintext

'the april uprising in bulgaria and its brutal suppression by the turks has caused outrage in the\nchancelleries of europe there is a risk that russia will take this as the excuse it seeks to engage\nthe ottomans and if they act and take constantinople then our trading routes to india will be under\nthreatat home gladstones pamphlet bulgarian horrors and the question of the east has stirred a\npublic appetite for action which could lead to support for intervention and make things difficult\nfor the prime minister he is faced with mortons fork if he supports action then it will be difficult\nto condemn russian interference if he counsels inaction then he risk appearing weak and callous at\nhome and abroad it may appear unfortunate that our political leaders are unable to agree on policy\nstrategy or tactics and it is true that this could lead to confusion about our aims but on\nreflection i think that the public disagreement between gladstone and disraeli presents an\nopportunity their 

In [186]:
ciphertext = playfair_encipher(plaintext, 'reichstag')
ciphertext

'beitnicknqecarqsrpmzqlsiakspkratshobgkbnxinirtarpogzbetfnhdaibgrbptrfnobistcrpbeihibqrcffcecrtwohoenoibeieictgecadbegahnavartxckfgkptfrctgtariiwhqtreatriftawtqsgbtfriwffwkbvdspkrofrixgegspfskpihpotaspaeopqktfriopnhseskrpscpnfttapevnakxekymghovniebeeigagaeufhlqsktaportxkkucmtfmzqlsiakurneensdspfsriunrtaepowobeiwittaibavtaceeiksqngmchkxoiaeftowisegepovrchreqqmfmitfsntnqqpesowecosiewroseppsvnkbfiberpbtkrkwkehqfgowesrinihkhfrprafterictdgirfxebefuespotdnepamertnqqpestgegeposriprfeckmgrfekkehqfgfweqvnhfvsnbarsprpftedierohiekrieqnotrdgrpgiaepoberoriecadkxoisirptyitpkvnigkyfqnbgaeufhspksshptkrbfgxkxoisinoowesnogatfibfwnhqpkcaeigkyfcskietgeinogsfcfwgbeitwoqqfchvgsegactwqesgiaergspkraetahntfibawberaeqqmfmitfsqepomoarpogspnfwnhkadbmzfwvstofcegepprberpfaibawbeiozmkcrlragbeihfroastfetrolqsktapoitvnkrdstikcnirtroatsppqqpesnoeawgricekranobihpomnegrfrpxkcdakfhosspfsrinirtdnhfpotaisfttawfriewcdfsrifewogirtwobeiwhfxaeigabertbktfhkhfnegkqcrobgtcksvnwcaohnfrosberakbxgkyfqzotapqenhirfxebekrgreiaepofwsewgp

In [189]:
playfair_decipher(ciphertext, 'reichstag')

'theapriluprisinginbulgariaanditsbrutalsupxpressionbytheturkshascausedoutrageinthechancelleriesofeuropethereisariskthatrusxsiawilltakethisastheexcuseitseekstoengagetheottomansandiftheyactandtakeconstantinoplethenourtradingroutestoindiawilxlbeunderthreatathomegladstonespamphletbulgarianhorrorsandthequestionofthexeasthasxstirredapublicappetiteforactionwhichcouldleadtosupportforinterventionandmakethingsdifxficultfortheprimeministerheisfacedwithmortonsforkifhesupportsactionthenitwillbedifficulttocondemnrussianinterferenceifhecounselsinactionthenheriskappearingweakandcalxlousathomeandabroaditmayappearunfortunatethatourpoliticalxleadersareunabletoagreeonpolicystrategyortacticsanditistruethatxthiscouldleadtoconfusionaboutouraimsbutonreflectionithinkthatxthepublicdisagreementbetweengladstoneanddisraelipresentsanopportunitytheirdisputeconductedinparliamentandthepressdemonstratestotheworldthetwofacesofthexempireatthesametimemorallyengagedandyetprudentthismayalxlowustoproceedwithdiscretiontotryto

In [206]:
key, score = playfair_simulated_annealing_break(ciphertext, fitness=Ptrigrams, workers=10, max_iterations=int(1e5), initial_temperature=500)
key, score

seidfrts tdeapsamupfttcndanculbfeiaingatregqtmhrqpxpscrtcon
napqbmkvyzgocxuseiftadkianrs atefpsovbpiorenfhazylfterainaebeiduaomqbcfpswereac
qyobxhweskuzpnvmlfgdapxfechdsnvzhbmyubuhiqmnj oscrprtwubwitibhinluymerthensttrekyodsrsotprirtiei
reicbstaigk creaprbguprisinginkulgartgansbtsadueagrupxpression
reichstarg theapriluprisinginbulgariaanditsbrutalsupxpression
reichstaig theapriluprisinginbulgariaanditsbrutalsupxpression
reifhst tfeapramuphtsinbinculbariaanditscrqtdgsqpxpression
stagbreicech theapriluprisinginhulcarxianditsbrutalsupapression
retarhsi itbeosblupaithnctndulcdstlbnfttedrpifgspwboshethon
reichstag theapriluprisinginbulgariaanditsbrutalsupxpression


({'alphabet': 'reichstaig',
  'letters_to_merge': {'j': 'i'},
  'pad_replace': False,
  'padding_letter': 'x'},
 -6173.17111571937)